In [285]:
import pandas as pd
import requests
import io
from bs4 import BeautifulSoup, NavigableString, Tag
from PIL import Image

In [286]:
# import existing excel file
plant_detail_df = pd.read_excel('../data/Garden_Plant.xlsx')

In [287]:
# set the header
plant_detail_df.columns = plant_detail_df.iloc[0]
plant_detail_df = plant_detail_df[1:]

In [288]:
# drop rows with no value
plant_detail_df = plant_detail_df.loc[~plant_detail_df['Name'].isna()]

In [289]:
# split the name column to name and variety
plant_detail_df[['plant_name', 'plant_variety']] = plant_detail_df['Name'].str.split(r'\n', expand = True)
plant_detail_df[['genus', 'species', 'subsp/var']] = plant_detail_df['Latin'].str.split(n = 2, expand = True)

In [290]:
# copy additional columns
plant_detail_df['seed_company'] = plant_detail_df['Company'].str.replace('\n', ' ')

In [293]:
plant_detail_df.reset_index(drop = True, inplace = True)

In [232]:
zip(plant_df['genus'], plant_df['species'], plant_df['subsp/var'])

In [236]:
' '.join(plant_df[['genus', 'species', 'subsp/var']])

'genus species subsp/var'

## def Function

In [210]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [ ]:
def get_url(search_kw, company)

    if company == "Johnny's Seed":
        url = f'https://www.johnnyseeds.com/search/?q={search_kw}&search-button=&lang=en_US'
        
        reqs = requests.get(link)
        soup = BeautifulSoup(reqs.text, 'html.parser')
        
        for x in soup.findAll('div', attrs = {'class': 'tile-name product-name'}):
            x.text.strip()
            
    elif company == "Baker Creek":
        url = f'https://www.rareseeds.com/catalogsearch/result/?q={search_kw}'
        
        reqs = requests.get(link)
        soup = BeautifulSoup(reqs.text, 'html.parser')
        
        for x in soup.findAll('div', attrs = {'class' : 'grid--item grid--item_product'}):
            x.find('span', attrs = {'class' : 'product--name'}).text.strip().split('\n')[0]
        
    elif company == "Botanical Interest":
        url = f'https://www.botanicalinterests.com/search?q={search_kw}&options%5Bprefix%5D=last'
        
        reqs = requests.get(link)
        soup = BeautifulSoup(reqs.text, 'html.parser')
        
        for x in soup.findAll('div', attrs = {'class' : 'card__content'}):
            x.find('a').text.strip()

In [304]:
def scrapping(plant, company, link):
    
    reqs = requests.get(link)
    soup = BeautifulSoup(reqs.text, 'html.parser')
    
    values = {}
    values.update({'plant_name' : plant,
                   'seed_company' : company})

    if company == "Johnny's Seed":
        try:   
            info = soup.findAll("dd", attrs = {'class' : "c-facts__definition"})
            for i in info:
                values.update({i.find_previous_sibling(name = 'dt').find('h3').text.strip() : i.text.strip()})

            detail = soup.find('div', attrs = {'id' : 'tabpanel_2-1'}).findAll('span', attrs = {'style' : ''})
            for i in detail:
                values.update({i.find_previous_sibling(name = 'span').text.strip() : i.text.strip()})
        except:
            pass
            
    elif company == "Baker Creek":
        try:
            values.update({'Latin Name' : soup.findAll('li')[-1].text.strip()})
        except:
            pass
        
    elif company == "Botanical Interest":
        try:
      
            imgs = soup.findAll('li', attrs = {'thumbnail-list__item slider__slide'})
            for i in imgs:
                if 'w_' in i.find('img')['src']:                
                    img_src = i.find('img')['src'].split('?')[0]

                    response = requests.get('http:' + img_src)
                    img = Image.open(io.BytesIO(response.content))
                    tts = pytesseract.image_to_string(img)
                    
                    values.update({'Latin Name' : tts.split('\n')[2:3]})
                    
            info = soup.find('div', attrs = {'class' : 'tab-content'}).findAll('p')
            for i in info:
                values.update({i.text.strip().split(': ')[0] : i.text.strip().split(': ')[1]})
        except:
            pass
            
    elif company == "Swallowtail Garden":
        try:
            values.update({'Latin Name' : soup.findAll('h3', attrs = {'class' : 'product__h3-botanical-names'})[0].text.strip()})
        except:
            pass
        
    result.append(values)

In [485]:
def sup_info(latin_name, matched):
    search_kw = '+'.join(latin_name.split()[0:1+1])
    
    url = f'https://plants.ces.ncsu.edu/find_a_plant/?q={search_kw}'
    reqs = requests.get(url)
    soup = BeautifulSoup(reqs.text, 'html.parser')
    
    values = {}
    output_lst = []
    
    output = soup.find('div', attrs = {'id' : 'plant_list'}).findAll('h2')
    for i in output:
        output_lst.append(i.find('a')['href'])
        
    if len(output_lst) == 1:
        matched = output_lst[0]

    elif len(output_lst) > 1:
        matched = difflib.get_close_matches(latin_name, output_lst)

    elif len(output_lst) < 1:
        pass
    
    result.update({latin_name : matched})

In [ ]:
    url = f'https://plants.ces.ncsu.edu{matched[0]}'
    reqs = requests.get(url)
    soup = BeautifulSoup(reqs.text, 'html.parser')
        
    element = soup.find('div', attrs = 'bricks').findAll("dd")
    for i in element:
        values.update({dd.find_previous_sibling(name = 'dt').text.strip() : dd.text.strip()})
        
    result.append(values)

In [ ]:
result = {}

for i in plant_df.loc[~plant_df['Latin Name'].isna()]['Latin Name']:
    sup_info(i, '')

In [ ]:
result

In [476]:
matched

['/plants/origanum-vulgare-subsp-hirtum/']

In [471]:
len(output_lst)

2

In [463]:
output_lst

['/plants/origanum-vulgare/', '/plants/origanum-vulgare-subsp-hirtum/']

In [464]:
len(output_lst)

2

In [465]:
matched = difflib.get_close_matches(latin_name, output_lst)

In [466]:
matched

['/plants/origanum-vulgare-subsp-hirtum/']

In [467]:
url = f'https://plants.ces.ncsu.edu{matched[0]}'

In [468]:
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')

In [469]:
element = soup.find('div', attrs = 'bricks').findAll("dd")

In [470]:
element

[<dd><span class="detail_display_attribute">Origanum</span></dd>,
 <dd><span class="detail_display_attribute">vulgare</span></dd>,
 <dd><span class="detail_display_attribute">Lamiaceae</span></dd>,
 <dd><span class="detail_display_attribute">Perennial</span></dd>,
 <dd><span class="detail_display_attribute">Greece, Turkey</span></dd>,
 <dd><span class="detail_display_attribute">Leaves are edible with a hot spicy taste and are used fresh or dried in soups, sauces, salads, stews, and dressings.  This is the oregano that is commonly purchased in stores in dried form.</span></dd>,
 <dd><span class="detail_display_attribute">Ground Cover</span></dd>,
 <dd><span class="detail_display_attribute">Herb</span></dd>,
 <dd><span class="detail_display_attribute">Herbaceous Perennial</span></dd>,
 <dd><span class="detail_display_attribute">Low</span></dd>,
 <dd><span class="detail_display_attribute">Full sun (6 or more hours of direct sunlight a day)</span></dd>,
 <dd><span class="detail_display_att

In [443]:
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')

In [445]:
values.update({'name':soup.find('div', attrs = {'id':'plant_detail'}).find('h1').text.strip()})
        

NameError: name 'element' is not defined

In [397]:
def sup_info(latin_name):

    for i in latin_name:
        search_kw = '+'.join(latin_name.split()[0:1+1])

        url = f'https://plants.ces.ncsu.edu/find_a_plant/?q={search_kw}'
        reqs = requests.get(url)
        soup = BeautifulSoup(reqs.text, 'html.parser')

        values = {}
        
        output = soup.find('div', attrs = {'id' : 'plant_list'}).findAll('h2')
        
        a = []

        for x in output:
            a.append(x.find('a')['href'])
           
            if len(a) == 1:
                url = f'https://plants.ces.ncsu.edu{a[0]}'
                reqs = requests.get(url)
                soup = BeautifulSoup(reqs.text, 'html.parser')
                
                for dd in soup.find('div', attrs = 'bricks').findAll("dd"):
                    values.update({'name' : soup.find('div', attrs = {'id' : 'plant_detail'}).find('h1').text.strip()})
                    values.update({dd.find_previous_sibling(name = 'dt').text.strip() : dd.text.strip()})
        result.append(values)

            elif len(a) < 1:
                pass

            elif len(a) > 1:
                close_match = difflib.get_close_matches(latin_name, a)

                url = f'https://plants.ces.ncsu.edu{close_match[0]}'
                reqs = requests.get(url)
                soup = BeautifulSoup(reqs.text, 'html.parser')

                for dd in soup.find('div', attrs = 'bricks').findAll("dd"):
                    values.update({'name' : soup.find('div', attrs = {'id' : 'plant_detail'}).find('h1').text.strip()})
                    values.update({dd.find_previous_sibling(name = 'dt').text.strip() : dd.text.strip()})
            result.append(values)

IndentationError: unexpected indent (800084708.py, line 29)

In [388]:
plant_df['Latin Name'] = plant_df['Latin Name'].str.replace('[^a-zA-Z0-9]', '')

C:\Users\kimbh\AppData\Local\Temp\ipykernel_2556\2522386161.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  plant_df['Latin Name'] = plant_df['Latin Name'].str.replace('[^a-zA-Z0-9]', '')


In [395]:
x

'plant_name'

In [394]:
for x in plant_df.loc[~plant_df['Latin Name'].isna()]:
    sup_info(x)

AttributeError: 'NoneType' object has no attribute 'findAll'

In [364]:
ref = pd.DataFrame.from_dict(result)

In [365]:
a

['/plants/origanum-vulgare/', '/plants/origanum-vulgare-subsp-hirtum/']

In [366]:
import difflib  
from difflib import SequenceMatcher  

In [369]:
if len(a) == 1:
    url = f'https://plants.ces.ncsu.edu{a[0]}'
    reqs = requests.get(url)
    soup = BeautifulSoup(reqs.text, 'html.parser')
    
elif len(a) < 1:
    pass

elif len(a) > 1:
    close_match = difflib.get_close_matches(latin_name, a)

In [370]:
close_match

['/plants/origanum-vulgare-subsp-hirtum/']

In [ ]:
if i.find(i[-1]) != -1:
    url = f'https://plants.ces.ncsu.edu{key}'
    reqs = requests.get(url)
    soup = BeautifulSoup(reqs.text, 'html.parser')

In [327]:
search_kw

'Mint'

In [315]:
reqs = requests.get('https://www.botanicalinterests.com/products/california-giants-zinnia-seeds')
soup = BeautifulSoup(reqs.text, 'html.parser')

In [316]:
imgs = soup.findAll('li', attrs = {'thumbnail-list__item slider__slide'})
for i in imgs:
    if 'w_' in i.find('img')['src']:                
        img_src = i.find('img')['src'].split('?')[0]

        response = requests.get('http:' + img_src)
        img = Image.open(io.BytesIO(response.content))
        tts = pytesseract.image_to_string(img)

        values.update({'Latin Name' : tts.split('\n')[2:3]})

info = soup.find('div', attrs = {'class' : 'tab-content'}).findAll('p')
for i in info:
    values.update({i.text.strip().split(': ')[0] : i.text.strip().split(': ')[1]})

In [320]:
tts

'ZINNIA\nCalifornia Giants Blend\n\nZinnia elegans\n\nANNUAL\nFULL SUN\nBlooms summer\nto frost\n24"-48" tall\n\nHEIRLOOM\n\nBold, beautiful,\n\n4"-6" flowers\n\nina rainbow\n\nof brilliant\n\ncolors last a\n\nvery long time\n\nin the garden\n\nand vase.\n\nSuperb fresh\n\ncut or dried .\nflower! i\n\nBotanical \\Nrerests.\n'

In [317]:
values

{'Latin Name': [''],
 'Days To Maturity': '365 Days',
 'Life Cycle': 'Biennial',
 'Hybrid Status': 'Open Pollinated',
 'Product Features': '',
 'DAYS TO GERMINATION:': '30 days.',
 'SOWING:': 'Transplant: Refrigerate the seeds until sowing. It is preferable to plant in the fall, but early spring plantings will also be successful. Tamp the seeds into the soil mix or just barely cover with soil and moisten. Seed requires light to germinate, along with alternating temperatures of warm and cold. Place the flats or trays outside where they will experience the needed temperature fluctuations. After 21 days, flats should be brought into warmer temperatures to germinate. Transplant seedlings outside in the spring when they are 3-4" tall, spacing seedlings 12-24" apart in rows that are 36" apart. Direct seed: Refrigerate the seeds until sowing. Sow in the fall or spring, 1/4- 3/8" deep, 10 seeds per foot in well-prepared seedbeds. Space plants 12-24" apart in each direction.',
 'LIGHT PREFERENC

In [97]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [305]:
result = []

a = plant_df.loc[~plant_df['link'].isna()]['plant_name']
b = plant_df.loc[~plant_df['link'].isna()]['seed_company']
c = plant_df.loc[~plant_df['link'].isna()]['link']

for x, y, z in zip(a, b, c):
    scrapping(x, y, z)

In [306]:
plant_df = pd.DataFrame.from_dict(result)

In [357]:
plant_df.loc[plant_df['seed_company'].str.contains("Seed")]

,plant_name,seed_company,Latin Name,Days to Maturity,Family,Native,Hardiness,Plant Dimensions,Variety Info,When to Sow Outside,...,DIRECT SEEDING:,AVG. DIRECT SEEDING RATE:,INSECT PESTS:,DISEASE:,POST HARVEST:,NOTE:,Direct seed:,DISEASES:,PELLETED SEED:,Direct seeding is not recommended.
6,Oregano,Johnny's Seed,Origanum vulgare hirtum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Thyme,Johnny's Seed,Thymus vulgaris,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Asclepias,Johnny's Seed,Asclepias curassavica,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,Tomato,Johnny's Seed,Solanum lycopersicum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,Chive,Johnny's Seed,Allium schoenoprasum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,Chive,Johnny's Seed,Allium tuberosum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,Sage,Johnny's Seed,Salvia officinalis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,Winter Savory,Johnny's Seed,Satureja montana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,Grass,Johnny's Seed,Melinis nerviglumis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,Marigold,Johnny's Seed,Tagetes erecta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [307]:
plant_df.loc[plant_df['']]

,plant_name,seed_company,Latin Name,Days to Maturity,Family,Native,Hardiness,Plant Dimensions,Variety Info,When to Sow Outside,...,DIRECT SEEDING:,AVG. DIRECT SEEDING RATE:,INSECT PESTS:,DISEASE:,POST HARVEST:,NOTE:,Direct seed:,DISEASES:,PELLETED SEED:,Direct seeding is not recommended.
0,Cardoon,Baker Creek,Cynara cardunculus var. scolymus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Artichoke,Botanical Interest,[],150–180 days,Asteraceae,Mediterranean region,Frost-tolerant perennial in USDA zones 7-10,"5' tall, 4'–8' wide",Dark green with slightly purple base. Heavy be...,Cold Climates,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Celery,Baker Creek,Apium graveolens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Butterfly Bush,Baker Creek,June Favorites,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Scallion,Botanical Interest,[],65 days,Alliaceae,Likely originated in eastern Asia,"Allium fistulosum is perennial in USDA 6-9, al...",NaN,"16""–18"" slender, white scallions with bluish-g...",4 to 6 weeks before your average last frost da...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Alpine Aster,Swallowtail Garden,Aster alpinus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,Lavender,Johnny's Seed,Lavandula angustifolia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
90,Mint,Botanical Interest,[Mentha sp.],NaN,Laminaceae,Unknown,"Perennial in USDA zones 6 and warmer, and poss...","18"" tall and spreading indefinitely by undergr...","This common mint is neither peppermint, spearm...",1 to 2 weeks after your average last frost dat...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,Shiso,Johnny's Seed,Perilla frutescens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def scrapping(plant, company, link):
    
    reqs = requests.get(link)
    soup = BeautifulSoup(reqs.text, 'html.parser')
    
    values = {}
    values.update({'plant_name' : plant,
                   'seed_company' : company})

    if company == "Johnny's Seed":
        try:   
            info = soup.findAll("dd", attrs = {'class' : "c-facts__definition"})
            for i in info:
                values.update({i.find_previous_sibling(name = 'dt').find('h3').text.strip() : i.text.strip()})

            detail = soup.find('div', attrs = {'id' : 'tabpanel_2-1'}).findAll('span', attrs = {'style' : ''})
            for i in detail:
                values.update({i.find_previous_sibling(name = 'span').text.strip() : i.text.strip()})
            
        except:
            pass
            
    elif company == "Baker Creek":
        try:
            values.update({'Latin Name' : soup.find('div', attrs = {'class' : 'value'}).findAll('li')[-1].text.strip()})
        
        except:
            pass
        
    elif company == "Botanical Interest":
        
        try:
            imgs = soup.findAll('li', attrs = {'thumbnail-list__item slider__slide'})
            info = soup.find('div', attrs = {'class' : 'tab-content'}).findAll('p')

            for x in imgs:
                if 'w_' in x.find('img')['src']:                
                    img_src = x.find('img')['src'].split('?')[0]

                    response = requests.get('http:' + img_src)
                    img = Image.open(io.BytesIO(response.content))
                    tts = pytesseract.image_to_string(img)

            for x in info:
                values.update({x.text.strip().split(': ')[0] : x.text.strip().split(': ')[1]})
        
        except:
            pass
            
    elif company == "Swallowtail Garden":

        try:
            values.update({'Latin_name' : soup.find('h3', attrs = {'class' : 'product__h3-botanical-names'}).text.strip()})
            
        except:
            pass
        
    result.append(values)